In [1]:
import os
import math

import scipy as sp
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import sklearn
from sklearn import cluster
from sklearn import neighbors

import torch
import torch.nn as nn
import torch.optim as optim

import scikit_wrappers
import sys
sys.path.append('/home/simon/Desktop/Code/functionlearning-contrastive')
from GP_utils import *

In [2]:
cuda = False
if torch.cuda.is_available():
    print("Using CUDA...")
    cuda = True

# GPU number
gpu = 0

Using CUDA...


In [3]:
def rand_rescale(inp, min_ht=.8):
    # rescale each curve to lie completely the interval [0,1], with difference between min and max at least min_ht

    new_mins = np.random.uniform(size=len(inp), low=0, high=(1 - min_ht) / 2)
    new_maxs = np.random.uniform(size=len(inp), low=(1 + min_ht) / 2, high=1)

    # first rescale so min=0 and max=1
    outp = inp - np.min(inp, axis=1)[:, None]
    outp = outp / np.max(outp, axis=1)[:, None]

    outp = outp * (new_maxs - new_mins)[:, None] + new_mins[:, None]
    return outp


In [4]:
train = []
total_curves=50000*200
xs=np.linspace(0,10,100)
for _ in range(total_curves//100):
    K, n = sample_comp_kernel() if np.random.rand() < .5 else sample_mix_kernel()
    n = 10 ** -5

    y1 = sample_gaussian(100, C=K.cov(xs, xs) + n * np.eye(len(xs)), use_svd=True)
    train.append(y1)
train = np.concatenate(train, axis=0)


In [5]:

train =rand_rescale(train)
train=train[:,None,:]

In [6]:
hyperparameters = {
    "batch_size": 10,
    "channels": 40,
    "compared_length": None,
    "depth": 10,
    "nb_steps": 1,
    "in_channels": 1,
    "kernel_size": 3,
    "penalty": None,
    "early_stopping": None,
    "lr": 0.001,
    "nb_random_samples": 10,
    "negative_penalty": 1,
    "out_channels": 320,
    "reduced_size": 160,
    "cuda": cuda,
    "gpu": gpu
}

In [7]:
encoder_yearly = scikit_wrappers.CausalCNNEncoderClassifier()
encoder_yearly.set_params(**hyperparameters)

CausalCNNEncoderClassifier(batch_size=10, channels=40, compared_length=inf,
                           cuda=True, depth=10, kernel_size=3, nb_steps=1,
                           out_channels=320, penalty=None, reduced_size=160)

In [ ]:
import time
STARTTIME=time.time()
total_iters=len(train)//512
for ii in range(13786,total_iters):
    ids=np.random.choice(len(train),512,replace=False)
    encoder_yearly.fit_encoder(train[ids], save_memory=True, verbose=True)
    elapsed=time.time()-STARTTIME
    print(ii/total_iters)

Epoch:  1
0.7058522349086068
Epoch:  1
0.7059034355639752
Epoch:  1
0.7059546362193436
Epoch:  1
0.706005836874712
Epoch:  1
0.7060570375300804
Epoch:  1
0.7061082381854488
Epoch:  1
0.7061594388408171
Epoch:  1
0.7062106394961856
Epoch:  1
0.7062618401515539
Epoch:  1
0.7063130408069224
Epoch:  1
0.7063642414622907
Epoch:  1
0.706415442117659
Epoch:  1
0.7064666427730275
Epoch:  1
0.7065178434283959
Epoch:  1
0.7065690440837643
Epoch:  1
0.7066202447391327
Epoch:  1
0.706671445394501
Epoch:  1
0.7067226460498695
Epoch:  1
0.7067738467052378
Epoch:  1
0.7068250473606063
Epoch:  1
0.7068762480159746
Epoch:  1
0.7069274486713429
Epoch:  1
0.7069786493267114
Epoch:  1
0.7070298499820797
Epoch:  1
0.7070810506374482
Epoch:  1
0.7071322512928165
Epoch:  1
0.7071834519481849
Epoch:  1
0.7072346526035533
Epoch:  1
0.7072858532589217
Epoch:  1
0.7073370539142901
Epoch:  1
0.7073882545696585
Epoch:  1
0.7074394552250268
Epoch:  1
0.7074906558803953
Epoch:  1
0.7075418565357636
Epoch:  1
0.70759

In [9]:
ii

13786

In [10]:
y0s=[]
labs=[]
for k_id in range(14):

  K,n=sample_comp_kernel(k_i=k_id) if k_id!=13 else sample_mix_kernel()
  y0=sample_gaussian(5000,C=K.cov(xs,xs)+n*np.eye(len(xs)))
  y0s.append(y0)
  labs=labs+[k_id]*len(y0)

y0s=np.concatenate(y0s,0)
labs=np.array(labs)
rp=np.random.permutation(len(y0s))
y0s=y0s[rp]
labs=labs[rp]


In [11]:
test=rand_rescale(y0s)[:,None,:]

In [12]:
xx = encoder_yearly.encode(test,batch_size=10)


In [13]:
xx.shape

(70000, 320)

In [14]:
from sklearn.linear_model import SGDClassifier
clf=SGDClassifier(alpha=10**-6)

In [15]:
clf.fit(xx[:len(test)//2],labs[:len(test)//2])

SGDClassifier(alpha=1e-06)

In [16]:
clf.score(xx[len(test)//2:],labs[len(test)//2:len(test)])

0.8049714285714286